In [1]:
import tensorflow as tf
import numpy as np

In [3]:
xy = np.loadtxt("train.txt", unpack=True)
x_data = np.transpose(xy[:-1])
y_data = np.reshape(xy[-1], (4, 1))

X = tf.placeholder(tf.float32, name ='X_input')
Y = tf.placeholder(tf.float32, name ='Y_input')

W1 = tf.Variable(tf.random_uniform([2, 2], -1.0, 1.0), name='weight1')
W2 = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0), name='weight2')

b1 = tf.Variable(tf.zeros([2]), name='Bias1')
b2 = tf.Variable(tf.zeros([1]), name='Bias2')

with tf.name_scope("layer2") as scope:
    L2 = tf.sigmoid(tf.matmul(X, W1) + b1)

with tf.name_scope("layer3") as scope:
    hypothesis = tf.sigmoid(tf.matmul(L2, W2) + b2)

with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    cost_summ =tf.summary.scalar("cost",cost)
    
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(cost)

w1_hist = tf.summary.histogram('weights1', W1)
w2_hist = tf.summary.histogram('weights2', W2)
b1_hist = tf.summary.histogram('biases1', b1)
b2_hist = tf.summary.histogram('biases2', b2)
y_hist =tf.summary.histogram('y',Y)
    
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    merged =tf.summary.merge([cost_summ,w1_hist,w2_hist,b1_hist,b2_hist,y_hist])
    writer =tf.summary.FileWriter('./logs/xor_logs',sess.graph)

    for step in range(20000):
        sess.run(train, feed_dict={X:x_data, Y: y_data})
        if step % 2000 == 0:
            summary = sess.run(merged, feed_dict={X:x_data, Y:y_data})
            writer.add_summary(summary, step)
    
    #correct_prediction = tf.equal(tf.floor(hypothesis+0.5), Y)
    #accuracy =tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    #param = [hypothesis,tf.floor(hypothesis+0.5),correct_prediction, accuracy]
    #result = sess.run(param, feed_dict={X:x_data, Y:y_data})
    
    #print(*result[0])
    #print(*result[1])
    #print(*result[2])
    #print(result[-1])
    #print('Accuracy:', accuracy.eval({X:x_data, Y:y_data}))

